# Instructions

This notebook shows you basic commands for either running a worker that trains other peoples models (Section 1: Run Worker) or to train your own model on the grid using the client (Section 2: Run Client). Before running this notebook, you need to have:

- installed IPFS (https://ipfs.io/docs/install/)
- run the command `ipfs daemon --enable-pubsub-experiment`
- run `python3 setup.py install` from the root directory of the OpenMined/Grid project (this project)

Then you're ready to run this notebook!

In [ ]:
from grid import ipfsapi
import base64
import random
import torch
import keras
import json
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD
import numpy as np 
from grid.pubsub.client import Client

In [ ]:
grid = Client()
grid.id

# Run Worker

In [ ]:
# grid.work()

# Run Client

In [ ]:
input = np.array([[0,0],[0,1],[1,0],[1,1]])
target = np.array([[0],[1],[1],[0]])

model = Sequential()
model.add(Dense(8, input_dim=2))
model.add(Activation('tanh'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

sgd = SGD(lr=0.1)
model.compile(loss='binary_crossentropy', optimizer=sgd)

In [ ]:
grid.api.dht_findpeer("QmXUyxfoi6bQbTdHXx8KftykpmHT66mBw1hKMCigPYVsv1")

In [ ]:
grid.api.pubsub_peers('openmined')

In [ ]:
grid.api.ping("QmXUyxfoi6bQbTdHXx8KftykpmHT66mBw1hKMCigPYVsv1")

In [ ]:
grid.id

In [ ]:
from grid import ipfsapi
import base64
import random
import torch
import keras
import json
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD
import numpy as np 
from grid.pubsub.client import Client

g = Client()

input = np.array([[0,0],[0,1],[1,0],[1,1]])
target = np.array([[0],[1],[1],[0]])

model = Sequential()
model.add(Dense(8, input_dim=2))
model.add(Activation('tanh'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

sgd = SGD(lr=0.1)
model.compile(loss='binary_crossentropy', optimizer=sgd)

model,train_spec = g.fit(model,input,target,epochs=200,log_interval=10)

In [ ]:
train_spec

In [ ]:
len(grid.api.pubsub_peers()['Strings'])

In [ ]:
model.summary()

# Run Worker

In [ ]:
grid.listen_to_channel(grid.fit_worker,'openmined_new_model')

In [ ]:
model.fit(X, y, batch_size=1, nb_epoch=1000)
print(model.predict_proba(X))

In [ ]:
def process_message(decoded):
    # send some sort of message
    try:
        # if message is a number... reply with the number + 1
        api.pubsub_pub(topic='openmined_model_1',payload=int(decoded['data']) + 1)
    except:
        print("could not accumulate... message not an int")
    print(decoded['data'])

# connect


my_id = ""
secret = random.randint(0,1000000)

# send secret identity


results = list()

for encoded in g:
    
    if('from' in encoded):
        # decode message
        decoded = {}
        decoded['from'] = base64.standard_b64decode(encoded['from'])
        decoded['data'] = base64.standard_b64decode(encoded['data']).decode('ascii')
        decoded['seqno'] = base64.standard_b64decode(encoded['seqno'])
        decoded['topicIDs'] = encoded['topicIDs']
        decoded['encoded'] = encoded
        results.append(decoded)

        # If I don't know who I am - wait for id
        if(my_id == ""):
            if(str(decoded['data'].split(":")[-1]) == str(secret)):
                my_id = str(decoded['from'])
                print("Id:" + my_id)

        # if I know who i am - process message
        else:

            if(str(decoded['from']) != my_id and decoded['data'][0:3] != 'id:'):
                process_message(decoded)
